#### Project Overview
This project is a data analysis of an online sports retail company's product data. The main objective is to analyze various product attributes—such as pricing, reviews, descriptions, and revenue—to provide recommendations that can help the company optimize its revenue.

#### Dataset and Tools
The analysis is based on a dataset that was originally composed of five separate .csv files from Kaggle.
- Data Source: The data has been loaded and combined into a single, in-memory SQLite database within this notebook.
- Tools Used: This project uses Python libraries, including pandas for data manipulation and sqlite3 for running SQL queries on the in-memory database.

#### Key Analysis Tasks
1. Count the total number of products, along with the number of non-missing values in description, listing_price, and last_visited
2. Find out how listing_price varies between Adidas and Nike products.
3. Create labels for products grouped by price range and brand.
4. Calculate the average discount offered by brand.
5. Calculate the correlation between reviews and revenue.
6. Split description into bins in increments of one hundred characters, and calculate average rating by for each bin.
7. Count the number of reviews per brand per month.
8. Create the footwear CTE, then calculate the number of products and average revenue from these items.
9. Copy the code used to create footwear then use a filter to return only products that are not in the CTE.

In [46]:
# %pip install pandas sqlite3

In [47]:
# Load Data and Create the Database
import pandas as pd
import sqlite3
import os

# Define the path to your dataset folder
dataset_path = 'Dataset'

# Load the datasets from the 'Dataset' folder
brands = pd.read_csv(os.path.join(dataset_path, 'brands_v2.csv'))
info = pd.read_csv(os.path.join(dataset_path, 'info_v2.csv'))
finance = pd.read_csv(os.path.join(dataset_path, 'finance.csv'))
traffic = pd.read_csv(os.path.join(dataset_path, 'traffic_v3.csv'))
reviews = pd.read_csv(os.path.join(dataset_path, 'reviews_v2.csv'))

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Load each DataFrame into a table in the database
brands.to_sql('brands', conn, index=False, if_exists='replace')
info.to_sql('info', conn, index=False, if_exists='replace')
finance.to_sql('finance', conn, index=False, if_exists='replace')
traffic.to_sql('traffic', conn, index=False, if_exists='replace')
reviews.to_sql('reviews', conn, index=False, if_exists='replace')

print("Data loaded into SQLite database successfully.")

Data loaded into SQLite database successfully.


In [48]:
query1 = """
SELECT
    COUNT(B.product_id) AS total_products,
    COUNT(I.description) AS count_description,
    COUNT(F.listing_price) AS count_listing_price,
    COUNT(T.last_visited) AS count_last_visited
FROM brands AS B
JOIN info AS I ON B.product_id = I.product_id
JOIN finance AS F ON B.product_id = F.product_id
JOIN traffic AS T ON B.product_id = T.product_id
WHERE B.brand IS NOT NULL;
"""
result1 = pd.read_sql(query1, conn)
print("Task 1: Missing Value Counts")
print(result1)

Task 1: Missing Value Counts
   total_products  count_description  count_listing_price  count_last_visited
0            3120               3117                 3120                2876


In [49]:
query2 = """
SELECT
    B.brand,
    COUNT(F.product_id) AS ProductsPerBrand,
    AVG(F.listing_price) AS AvgPrice,
    MIN(F.listing_price) AS LowestPrice,
    MAX(F.listing_price) AS HighestPrice
FROM brands AS B
JOIN finance AS F ON B.product_id = F.product_id
WHERE F.listing_price > 0 AND B.brand IS NOT NULL
GROUP BY B.brand
ORDER BY B.brand;
"""
result2 = pd.read_sql(query2, conn)
print("\nTask 2: Nike vs. Adidas Pricing")
print(result2)


Task 2: Nike vs. Adidas Pricing
    brand  ProductsPerBrand    AvgPrice  LowestPrice  HighestPrice
0  Adidas              2575   75.725146         8.99        299.99
1    Nike               191  117.970942        29.95        199.95


In [50]:
query3 = """
SELECT
    B.brand,
    F.sale_price,
    CASE
        WHEN F.sale_price <= 50 THEN 'Low'
        WHEN F.sale_price BETWEEN 51 AND 100 THEN 'Medium'
        WHEN F.sale_price BETWEEN 101 AND 250 THEN 'High'
        ELSE 'Very High'
    END AS price_range_label
FROM finance AS F
JOIN brands AS B ON F.product_id = B.product_id
WHERE B.brand IS NOT NULL;
"""
result3 = pd.read_sql(query3, conn)
print("\nTask 3: Price Range Labels")
print(result3)


Task 3: Price Range Labels
       brand  sale_price price_range_label
0     Adidas       37.99               Low
1     Adidas        5.99               Low
2     Adidas       34.99               Low
3     Adidas       39.99               Low
4     Adidas       19.20               Low
...      ...         ...               ...
3115    Nike       64.95            Medium
3116    Nike      139.95              High
3117    Nike      127.97              High
3118    Nike      169.95              High
3119    Nike       62.97            Medium

[3120 rows x 3 columns]


In [51]:
query4 = """
SELECT
    B.brand,
    AVG((F.listing_price - F.sale_price) * 1.0 / F.listing_price) * 100 AS AvgDiscount
FROM brands AS B
JOIN finance AS F ON B.product_id = F.product_id
WHERE F.listing_price > 0 AND B.brand IS NOT NULL
GROUP BY B.brand
ORDER BY B.brand;
"""
result4 = pd.read_sql(query4, conn)
print("\nTask 4: Average Discount by Brand")
print(result4)


Task 4: Average Discount by Brand
    brand  AvgDiscount
0  Adidas    33.458081
1    Nike    28.552710


In [52]:
query5 = """
SELECT
    (COUNT(*) * SUM(R.reviews * F.revenue) - SUM(R.reviews) * SUM(F.revenue)) /
    (SQRT(COUNT(*) * SUM(R.reviews * R.reviews) - POWER(SUM(R.reviews), 2)) *
     SQRT(COUNT(*) * SUM(F.revenue * F.revenue) - POWER(SUM(F.revenue), 2))) AS Corr_Reviews_Revenue
FROM reviews AS R
JOIN finance AS F ON R.product_id = F.product_id
JOIN brands AS B ON B.product_id = F.product_id
WHERE B.brand IS NOT NULL;
"""
result5 = pd.read_sql(query5, conn)
print("\nTask 5: Correlation between Reviews and Revenue")
print(result5)


Task 5: Correlation between Reviews and Revenue
   Corr_Reviews_Revenue
0              0.651851


In [53]:
query6 = """
SELECT
    CAST(CEILING(LENGTH(I.description) * 1.0 / 100) AS INTEGER) AS Bins,
    AVG(R.rating) AS AvgRating
FROM info AS I
JOIN reviews AS R ON I.product_id = R.product_id
WHERE I.description IS NOT NULL
GROUP BY Bins
ORDER BY Bins;
"""
result6 = pd.read_sql(query6, conn)
print("\nTask 6: Average Rating by Description Length Bin")
print(result6)


Task 6: Average Rating by Description Length Bin
   Bins  AvgRating
0     1   2.257143
1     2   3.185551
2     3   3.277815
3     4   3.285407
4     5   3.345763
5     6   3.120000
6     7   3.653333


In [54]:
query7 = """
SELECT
    B.brand,
    STRFTIME('%m', T.last_visited) AS Month,
    SUM(R.reviews) AS TotalReviews
FROM brands AS B
JOIN reviews AS R ON B.product_id = R.product_id
JOIN traffic AS T ON B.product_id = T.product_id
WHERE T.last_visited IS NOT NULL AND B.brand IS NOT NULL
GROUP BY B.brand, Month
ORDER BY B.brand, Month;
"""
result7 = pd.read_sql(query7, conn)
print("\nTask 7: Reviews by Month and Brand")
print(result7)


Task 7: Reviews by Month and Brand
     brand Month  TotalReviews
0   Adidas    01       11852.0
1   Adidas    02       13372.0
2   Adidas    03       12987.0
3   Adidas    04        9145.0
4   Adidas    05        7358.0
5   Adidas    06        7773.0
6   Adidas    07        8564.0
7   Adidas    08        9813.0
8   Adidas    09        9246.0
9   Adidas    10       10095.0
10  Adidas    11        6999.0
11  Adidas    12        9099.0
12    Nike    01         307.0
13    Nike    02         357.0
14    Nike    03         577.0
15    Nike    04         277.0
16    Nike    05         592.0
17    Nike    06         259.0
18    Nike    07         422.0
19    Nike    08         147.0
20    Nike    09         202.0
21    Nike    10         197.0
22    Nike    11         226.0
23    Nike    12         317.0


In [55]:
query8 = """
WITH footwear AS (
    SELECT product_id
    FROM info
    WHERE description LIKE '%shoe%' OR description LIKE '%trainer%' OR description LIKE '%foot%'
)
SELECT
    COUNT(*) AS ProductCount,
    AVG(F.revenue) AS AvgRevenue
FROM footwear AS FW
JOIN finance AS F ON FW.product_id = F.product_id;
"""
result8 = pd.read_sql(query8, conn)
print("\nTask 8: Footwear Product Performance")
print(result8)


Task 8: Footwear Product Performance
   ProductCount   AvgRevenue
0          2700  4235.461315


In [56]:
query9 = """
WITH footwear AS (
    SELECT product_id
    FROM info
    WHERE description LIKE '%shoe%' OR description LIKE '%trainer%' OR description LIKE '%foot%'
)
SELECT
    COUNT(*) AS ProductCount,
    AVG(F.revenue) AS AvgRevenue
FROM info AS I
JOIN finance AS F ON I.product_id = F.product_id
LEFT JOIN footwear AS FW ON FW.product_id = I.product_id
WHERE FW.product_id IS NULL AND I.description IS NOT NULL;
"""
result9 = pd.read_sql(query9, conn)
print("\nTask 9: Clothing Product Performance")
print(result9)

conn.close()


Task 9: Clothing Product Performance
   ProductCount   AvgRevenue
0           417  2080.244964
